Move data in to correct directories

In [1]:
#Make sure we are in the right directory
%pwd

u'/home/ubuntu/nbs/hw1'

In [2]:
import os, sys
current_dir = os.getcwd()
current_dir
HOME_DIR = current_dir
DATA_DIR = HOME_DIR + '/data/redux'

In [3]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))

#import modules
from glob import glob
import numpy as np
from vgg16 import Vgg16

Using Theano backend.


## Action Plan
1. Create Validation and Sample sets
2. Rearrange image files into their respective directories

In [4]:
%cd $DATA_DIR
%mkdir -p valid
%mkdir -p results
%mkdir -p sample/train
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p sample/test
%mkdir -p test/unknown

/home/ubuntu/nbs/hw1/data/redux


In [5]:
%cd $DATA_DIR/train

/home/ubuntu/nbs/hw1/data/redux/train


In [6]:
# Move files into validation folders
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(2000): os.rename(shuf[i], DATA_DIR+'/valid/' + shuf[i])

In [7]:
from shutil import copyfile

In [8]:
# Copy files into sample folders
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(16): copyfile(shuf[i], DATA_DIR+'/sample/train/' + shuf[i])

In [9]:
%cd $DATA_DIR/valid

/home/ubuntu/nbs/hw1/data/redux/valid


In [10]:
g = glob('*.jpg')
shuf = np.random.permutation(g)
for i in range(8): copyfile(shuf[i], DATA_DIR+'/sample/valid/' + shuf[i])

## Arrange files into their own classes

In [11]:
#Divide cat/dog images into separate directories

%cd $DATA_DIR/sample/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_DIR/sample/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_DIR/valid
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

%cd $DATA_DIR/train
%mkdir cats
%mkdir dogs
%mv cat.*.jpg cats/
%mv dog.*.jpg dogs/

/home/ubuntu/nbs/hw1/data/redux/sample/train
/home/ubuntu/nbs/hw1/data/redux/sample/valid
/home/ubuntu/nbs/hw1/data/redux/valid
/home/ubuntu/nbs/hw1/data/redux/train


In [13]:
# Create single 'unknown' class for test set
%cd $DATA_DIR/test
%mv *.jpg unknown/

/home/ubuntu/nbs/hw1/data/redux/test


## Apply finetuning to dataset

In [14]:
%cd $DATA_DIR

train_path = DATA_DIR + '/sample/train'
#train_path = DATA_DIR + 'train'
valid_path = DATA_DIR + '/valid'
test_path = DATA_DIR + '/test'
results_path = DATA_DIR + '/results'

/home/ubuntu/nbs/hw1/data/redux


In [15]:
# Load vgg16
vgg = Vgg16()

32768/35363 [==========================>...] - ETA: 0s

In [17]:
batch_size = 64
num_epochs = 1

In [ ]:
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size)
vgg.finetune(batches)

In [19]:
# Fit the data and validate against valid set
for epoch in range(num_epochs):
    print "Running epoch: %d" % epoch
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename = 'ft%d.h5' % epoch
    vgg.model.save_weights(results_path+latest_weights_filename)
print "Ran %d fit operations" % num_epochs

IndentationError: expected an indented block (<ipython-input-19-807a645e9140>, line 2)

## Generate predictions

In [ ]:
# Predict all images in test dataset
test_batches, preds = vgg.test(test_path, batch_size=batch_size*2)

In [20]:
# Verify data looks ok
print preds[:5]
print test_batches.filenames[:5]

NameError: name 'preds' is not defined

In [22]:
# Check 2 images
from PIL import Image
Image.open(test_path + '/' + test_batches.filenames[2])
Image.open(test_path + '/' + test_batches.filenames[4])

In [23]:
# Save predicted results
import bcolz

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()

def load_array(fname):
    return bcolz.open(fname)[:]

preds_filename = results_path + '/test_preds.dat'
test_filenames_filename = results_path + '/test_filenames.dat'
save_array(preds_filename, preds)
save_array(test_filenames_filename, test_batches.filenames)

## Kaggle Submission

In [ ]:
preds = load_array(preds_filename)
filenames = load_array(test_filenames_filename)

In [ ]:
# Find the "dog" column in preds (could be preds[:0] or preds[:1])
isdog = preds[:0]



In [ ]:
# Clip high and low probability scores (for log-loss sake)
isdog = isdog.clip(min=0.5, max=0.95)

In [ ]:
# Get image IDs for each file
ids = np.array([int(f[8:f.find(".")]) for f in filenames])

In [ ]:
# Stack preds and ids
subm = np.stack((ids, isdog), axis=1)
subm[:5]

In [ ]:
# Create output file for submission
%cd $DATA_DIR
submission_file_name = 'submission1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')